# <center> InternAnalytics </center> #

### 1. Introduction ###
Objectif :
Le projet InternAnalytics a pour but d’analyser une base de données regroupant des offres d'emploi afin de tirer des insights exploitables pour :

- Comprendre les tendances du marché de l’emploi (secteurs les plus actifs, localisations recherchées, etc.).
- Identifier les facteurs d’attractivité des offres d’emploi (entreprises populaires, offres les plus vues).
- Évaluer les tendances salariales en fonction des secteurs, des localisations, et des postes proposés.
- Offrir des recommandations exploitables, par exemple :
    -  Aux chercheurs d’emploi : quels secteurs et localisations privilégier.
    - Aux recruteurs : comment optimiser leurs offres pour attirer plus de candidats.
- S’informer sur les offres d’alternance :
    - Identifier les entreprises qui proposent des contrats en alternance.
    - Analyser les postes et secteurs qui recrutent le plus en alternance.
    - Comprendre les salaires et avantages liés aux offres d’alternance.
- Champs disponibles : `job_id`, `company_name`, `title`, `description`, `max_salary`, `pay_period`, `location`, `company_id`, `views, med_salary`.

Ce projet vise donc à offrir une vision globale et spécifique des opportunités sur le marché, avec un focus particulier sur les alternances, pour mieux répondre aux besoins des étudiants et jeunes professionnels.


### 2. Chargement des données ###
- Importation des bibliothèques nécessaires (e.g., pandas, numpy, matplotlib, seaborn).
- Chargement du fichier de données (CSV, SQL, ou autre).
- Aperçu des données (df.head() et df.info()).

In [2]:
# Utility import
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset import
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

OSError: Could not find kaggle.json. Make sure it's located in /home/tom/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [47]:
# Dowload dataset using kaggle API
path = os.getcwd().replace("notebooks", "kaggle.json")

with open(path, 'r') as file:
    kaggle_creds = json.load(file)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset_name = 'arshkon/linkedin-job-postings'
download_path = "../data/raw"
os.makedirs(download_path, exist_ok=True)

api = KaggleApi()
api.authenticate()
print("Téléchargement en cours...")
api.dataset_download_files(dataset_name, path=download_path, unzip=True)
print(f"Dataset téléchargé et extrait dans : {download_path}")

Téléchargement en cours...
Dataset URL: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings
Dataset téléchargé et extrait dans : ../data/raw


### 3. Nettoyage et Préparation des Données ###
- Détection des données manquantes : Vérifier les colonnes avec des valeurs NaN ou vides.
- Traitement des doublons : Vérifier et supprimer les entrées redondantes si nécessaire.
- Standardisation des colonnes : Par exemple, uniformiser les formats des - salaires (max_salary, med_salary) ou des localisations (location).
- Conversion des types de données pour faciliter l’analyse :
    - pay_period en fréquence (mensuel, annuel, etc.).
views en entier.
    - Extraction de catégories pertinentes à partir de description (ex : analyse des mots-clés).

In [4]:
# Charger les donnée dans un dataframe pandas 
data = pd.read_csv("../data/raw/postings.csv")

In [5]:
data_clean = data.drop_duplicates()
data_clean = data_clean.drop(columns=['med_salary', 'min_salary', 'max_salary', 'listed_time', 'posting_domain', 'original_listed_time', 'job_posting_url', 'expiry', 'closed_time', 'application_url' ,'sponsored', 'zip_code', 'fips', 'compensation_type'])

In [6]:
data_clean['remote_allowed'] = data_clean['remote_allowed'].fillna(0).astype(int)
data_clean['applies'] = data_clean['applies'].fillna(0).astype(int)
for elt in data_clean :
    data_clean[elt] = data_clean[elt].fillna('Not specified')

data_clean = data_clean.dropna(subset=['company_name'])

os.makedirs("../data/interim/", exist_ok=True)
data_clean.to_csv("../data/interim/posting_clean.csv", header=True)

### 4. Exploration des Données (EDA - Exploratory Data Analysis) ###
- Statistiques descriptives :
    - Distribution des salaires (max_salary, med_salary).
    - Entreprises avec le plus d’offres (company_name).
- Visualisation des données :
    - Histogrammes et boxplots pour les salaires.
    - Nuage de points pour corréler les salaires et les vues.
    - Graphiques géographiques pour la localisation des offres (location).
- Analyse des tendances :
    - Tendances par secteur (basé sur les mots-clés dans title ou description).
    - Entreprises les plus attractives (basé sur views).

### 5. Analyses Avancées et Insights ###

- Segmentation des données :
    - Comparaison des salaires par localisation.
    - Entreprises par secteur d’activité.
- Détection des relations et corrélations :
    - Corrélation entre views et max_salary.
    - Analyse des types d’emploi les plus attractifs (title, description).
- Suggestions basées sur les insights :
    - Quels types d’emplois attirer le plus de vues ?
    - Quels secteurs offrent les meilleurs salaires ?